In [1]:
import pandas as pd
from datetime import datetime
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows

# 1. Pivot - PIVOT() example
- Here, we can create a pivot table to group together shops and items to see how each is performing.

In [2]:
shop_sales = [["New York","Shoes",3000],["New York","Jackets",4000],["London","Shoes",2000],["London","Jackets",5000]]
df_shop = pd.DataFrame(shop_sales, columns=["Shop","Item","Sales"])
df_shop

,Shop,Item,Sales
0,New York,Shoes,3000
1,New York,Jackets,4000
2,London,Shoes,2000
3,London,Jackets,5000


In [3]:
# See how many items are sold in each shop.
df_shop.pivot(index="Shop", columns="Item", values="Sales")

Item,Jackets,Shoes
Shop,,
London,5000,2000
New York,4000,3000


In [4]:
# Here we pivot the other way.
df_shop.pivot(index="Item", columns="Shop", values="Sales")

Shop,London,New York
Item,,
Jackets,5000,4000
Shoes,2000,3000


# 2. Pivot 2 - PIVOT() example with grouped sales by date

- Here we look at how many sales were performed per salesperson per date

In [5]:
df_dates_arr = [
    ["Dave",'2021-11-17',40],
    ["Sally",'2021-11-17',0],
    ["Bob",'2021-11-17',200],
    ["Dave",'2021-11-18',30],
    ["Sally",'2021-11-18',10],
    ["Bob",'2021-11-18',210],
    ["Dave",'2021-11-19',40],
    ["Sally",'2021-11-19',390],
    ["Bob",'2021-11-19',140]
]
df_dates = pd.DataFrame(df_dates_arr, columns=["Salesperson","Date","Sales"])
df_dates

,Salesperson,Date,Sales
0,Dave,2021-11-17,40
1,Sally,2021-11-17,0
2,Bob,2021-11-17,200
3,Dave,2021-11-18,30
4,Sally,2021-11-18,10
5,Bob,2021-11-18,210
6,Dave,2021-11-19,40
7,Sally,2021-11-19,390
8,Bob,2021-11-19,140


In [6]:
df_dates.pivot(index="Date", columns="Salesperson", values="Sales")

Salesperson,Bob,Dave,Sally
Date,,,
2021-11-17,200,40,0
2021-11-18,210,30,10
2021-11-19,140,40,390


In [7]:
# We can also pivot the other way with salesperson as the index
df_dates.pivot(index="Salesperson", columns="Date", values="Sales")

Date,2021-11-17,2021-11-18,2021-11-19
Salesperson,,,
Bob,200,210,140
Dave,40,30,40
Sally,0,10,390


# 3. Pivot - PIVOT_TABLE() - aggregated functions

- You can easily specify an aggreagate function in pandas pivot tables.

In [8]:
sales_data = [
    ["Credit Card","Member","Technology",200,],
    ["Gift Card","Normal","Health and Beauty",100],
    ["Cash","Normal","Technology",120],
    ["Credit Card","Normal","Technology",300],
    ["Credit Card","Member","Health and Beauty",200],
    ["Gift Card","Member","Health and Beauty",100],
    ["Credit Card","Normal","Garden",100],
    ["Cash","Member","Health and Beauty",50],
    ["Credit Card","Normal","Garden",60],
    ["Gift Card","Member","Technology",120],
    ["Cash","Normal","Technology",30],
    ["Gift Card","Deluxe","Health and Beauty",300]
]
df_sales_data = pd.DataFrame(sales_data, columns=["Payment Type","Membership Level","Department","Sales Total"])
df_sales_data

,Payment Type,Membership Level,Department,Sales Total
0,Credit Card,Member,Technology,200
1,Gift Card,Normal,Health and Beauty,100
2,Cash,Normal,Technology,120
3,Credit Card,Normal,Technology,300
4,Credit Card,Member,Health and Beauty,200
5,Gift Card,Member,Health and Beauty,100
6,Credit Card,Normal,Garden,100
7,Cash,Member,Health and Beauty,50
8,Credit Card,Normal,Garden,60
9,Gift Card,Member,Technology,120


In [9]:
# Payment type totals sorted
df_sales_data.pivot_table(index=["Payment Type"],values=["Sales Total"],aggfunc="sum").sort_values(by="Sales Total", ascending=False)

,Sales Total
Payment Type,
Credit Card,860
Gift Card,620
Cash,200


In [10]:
# Departments by sales - get top performing department
df_top_dept = df_sales_data.pivot_table(index=["Department"], values=["Sales Total"], aggfunc="sum").sort_values(by="Sales Total",ascending=False)
df_top_dept.head(1)

,Sales Total
Department,
Technology,770


In [11]:
# Combining groups of results
# Here we get the results based on department and payment type
by_dept_payment_type = df_sales_data.pivot_table(index=["Department"],columns=["Payment Type"],values=["Sales Total"],aggfunc=sum)
by_dept_payment_type

Sales Total                      
Payment Type             Cash Credit Card Gift Card
Department                                         
Garden                    NaN       160.0       NaN
Health and Beauty        50.0       200.0     500.0
Technology              150.0       500.0     120.0

- The above result has NaN where 0 sales were made in a department... so we can use fillna() again to tidy the results.
- We also want to return the totals as ints instead of floats...

In [12]:
by_dept_payment_type.fillna(0, inplace=True)
by_dept_payment_type["Sales Total"] = by_dept_payment_type["Sales Total"].astype(int)
by_dept_payment_type

Sales Total                      
Payment Type             Cash Credit Card Gift Card
Department                                         
Garden                      0         160         0
Health and Beauty          50         200       500
Technology                150         500       120

# 4. Pivot table example - FLOAT to 2 decimal places (percentages)

- Here we create a pivot table to see how the percentage each payment type accounted for.
- Then we create a column calculating percentage of sales and rounding down and converting to a string.

In [13]:
df_membership_level = df_sales_data.pivot_table(index=["Membership Level"],values=["Sales Total"],aggfunc="sum").sort_values(by="Sales Total", ascending=False)
df_membership_level

,Sales Total
Membership Level,
Normal,710
Member,670
Deluxe,300


In [14]:
# Create a new column with a percentage of sales 
df_membership_level['Percentage of sales'] = round((df_membership_level['Sales Total'] / df_membership_level['Sales Total'].sum() * 100),2).astype(str) + " %"
df_membership_level

,Sales Total,Percentage of sales
Membership Level,,
Normal,710,42.26 %
Member,670,39.88 %
Deluxe,300,17.86 %


# 5. openpyxl - Save Dataframes to different Excel workbook sheets

- Here we will save 2 dataframes to 2 separate sheets in zan Excel workbook.
- With openpyxl you can use the dataframe_to_rows function to copy a dataframe to a new sheet

In [15]:
# Create a workbook
wb = Workbook()

In [16]:
# Create a new sheet
department_and_payment_type = wb.create_sheet(title="Department and Payment Type")

In [17]:
# Here we append rows from the 'by_dept_payment_type' dataframe to the excel sheet
for r in dataframe_to_rows(by_dept_payment_type, index=True, header=False):
    department_and_payment_type.append(r)

In [18]:
# Create a new sheet
membership_level_sales_percentage = wb.create_sheet(title="Membership Sales Percentage")

In [19]:
# Here we append rows from the 'by_dept_payment_type' dataframe to the excel sheet
for r in dataframe_to_rows(df_membership_level, index=True, header=False):
    membership_level_sales_percentage.append(r)

- Finally, save the workbook as a workbook named 'sales_analysis.xlsx'.  The workbook should have the 2 dataframes above saved into the 2 sheets named above.

In [20]:
wb.save("sales_analysis.xlsx")